## Exploração dos dados de Falha

Aqui será realizada a análise exploratória e preparação dos dados de falha.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import openpyxl

In [2]:
df_falhas = pd.read_excel('../data/FALHAS_PREDICT 1.xlsx',skiprows=1, usecols='B:K')
df_falhas.to_csv('../data/FALHAS_PREDICT_1.csv', index=False)

/home/bielle/Documents/GitHub/2024-2A-T08-EC07-G01/.venv/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [3]:
# Intanciando o dataframe 

df_falhas = pd.read_csv('../data/FALHAS_PREDICT_1.csv')
df_falhas.head()


/tmp/ipykernel_3731/2438449128.py:3: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df_falhas = pd.read_csv('../data/FALHAS_PREDICT_1.csv')


,KNR,MODELO,COR,MOTOR,ESTACAO,USUARIO,HALLE,FALHA,S_GROUP_ID,DATA
0,2023-2056234,T-CROSS,2R2R,DHS,HCEQS506,4013334,ZP7,PARALAMA DIANTEIRO 01. AMASSADO,5,2024-05-20 21:31:51.992
1,2023-2056234,T-CROSS,2R2R,DHS,HCEQS506,4013334,ZP7,PORTA TRASEIRA 04. CAROÇO / PICO,4,2024-05-20 21:30:22.772
2,2023-2056234,T-CROSS,2R2R,DHS,PCEQS100,4004554,ZP5,PORTA DIANTEIRA SOLDA - RESPINGO,4,2023-12-06 06:48:29.543
3,2023-2056234,T-CROSS,2R2R,DHS,PCEQS114,2020297,ZP5,CINTO DE SEGURANÇA - FIXAÇÃO ASSOALHO ROSCA NOK,2,2023-12-05 21:36:33.284
4,2023-2056234,T-CROSS,2R2R,DHS,PCEQS620,2058570,ZP6,MÓDULO DO ACC (13) PEÇA INCORRETA ¨,9830946,2024-05-20 20:04:55.534


In [5]:
df_falhas.describe()

,KNR,MODELO,COR,MOTOR,ESTACAO,USUARIO,HALLE,FALHA,S_GROUP_ID,DATA
count,758916,758916,758916,758916,758906,758906,757311,758916,758916,758916
unique,87407,1,13,5,80,454,30,6581,19,585194
top,2024-2816497,T-CROSS,K2K2,DHS,HCEQS506,2039346,ZP7,Painel lateral 05. Sujeira na pintura,2,2023-10-05 14:27:31.000
freq,100,758916,197803,463239,381564,36311,306878,25630,231272,14


É possível notar que há dados nulos em algumas colunas, a julgar pela contagem

In [6]:
#Excluir linhas que possuem dados nulos
df_falhas = df_falhas.dropna()
df_falhas.describe()


,KNR,MODELO,COR,MOTOR,ESTACAO,USUARIO,HALLE,FALHA,S_GROUP_ID,DATA
count,757311,757311,757311,757311,757311,757311,757311,757311,757311,757311
unique,87405,1,13,5,78,451,30,6575,19,583929
top,2024-2816497,T-CROSS,K2K2,DHS,HCEQS506,2039346,ZP7,Painel lateral 05. Sujeira na pintura,2,2023-10-05 14:27:31.000
freq,100,757311,197462,462316,381564,36311,306878,25536,230807,14


In [7]:
# Ver o tipo das colunas e verificar se a retirada de dados nulos deu certo.
df_falhas.info()

<class 'pandas.core.frame.DataFrame'>
Index: 757311 entries, 0 to 758915
Data columns (total 10 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   KNR         757311 non-null  object
 1   MODELO      757311 non-null  object
 2   COR         757311 non-null  object
 3   MOTOR       757311 non-null  object
 4   ESTACAO     757311 non-null  object
 5   USUARIO     757311 non-null  object
 6   HALLE       757311 non-null  object
 7   FALHA       757311 non-null  object
 8   S_GROUP_ID  757311 non-null  object
 9   DATA        757311 non-null  object
dtypes: object(10)
memory usage: 63.6+ MB


In [8]:
#Checando quantos valores únicos de tipos de falha possuímos
df_falhas["S_GROUP_ID"].unique()

array(['5', '4', '2', '9830946', '1', '-2', '133', '140', '#MULTIVALUE',
       '137', 2, 4, 133, 1, 5, 9830946, -2, 140, 137], dtype=object)

## Percepções

Os dados não estão padronizados, ou seja, há o tipo 1 numérico e o tipo 1 string, então é necessário realizar essa padronização. Além disso, há a presença de Multivalues, que não sabemos a quais categorias pertencem.

In [9]:
print(df_falhas['S_GROUP_ID'].value_counts())

S_GROUP_ID
2              230807
4              129372
2               87828
1               83836
5               82127
4               44415
5               29709
1               27925
-2              25487
9830946          4213
133              3934
-2               3790
9830946          1405
133               914
140               895
140               409
137               122
137                76
#MULTIVALUE        47
Name: count, dtype: int64


Visto que há pouquíssimos registro de Multivalue em relação ao todo (apenas 47 ocorrências), é interessante remover os dados pois há uma falta de definição do significado dele. Isso faz com que evite acontecer erros no modelo, permitindo uma melhor precisão nele.

In [4]:
#Criando um novo DataFrame retirando as ocorrências que possuem #MULTIVALULES
df_tratado = df_falhas.drop(df_falhas[df_falhas['S_GROUP_ID'] == '#MULTIVALUE'].index)
df_tratado['S_GROUP_ID'] = df_tratado['S_GROUP_ID'].astype(int)
df_tratado["S_GROUP_ID"].unique()

array([      5,       4,       2, 9830946,       1,      -2,     133,
           140,     137])

In [5]:
print(df_tratado.dtypes)

KNR           object
MODELO        object
COR           object
MOTOR         object
ESTACAO       object
USUARIO       object
HALLE         object
FALHA         object
S_GROUP_ID     int64
DATA          object
dtype: object


## Exportação a primeira versão dos dados

Após esse primeiro tratamento dos dados, vamos exportar em csv a tabela criada, a fim de usá-la em outros notebooks.

In [6]:
df_tratado.to_csv('../data/Tabela_Falhas_Atualizada.csv', index=False)
df_atualizado = pd.read_csv('../data/Tabela_Falhas_Atualizada.csv')

In [13]:
#Visualização da tabela pós modificações
df_atualizado.head()

,KNR,MODELO,COR,MOTOR,ESTACAO,USUARIO,HALLE,FALHA,S_GROUP_ID,DATA
0,2023-2056234,T-CROSS,2R2R,DHS,HCEQS506,4013334,ZP7,PARALAMA DIANTEIRO 01. AMASSADO,5,2024-05-20 21:31:51.992
1,2023-2056234,T-CROSS,2R2R,DHS,HCEQS506,4013334,ZP7,PORTA TRASEIRA 04. CAROÇO / PICO,4,2024-05-20 21:30:22.772
2,2023-2056234,T-CROSS,2R2R,DHS,PCEQS100,4004554,ZP5,PORTA DIANTEIRA SOLDA - RESPINGO,4,2023-12-06 06:48:29.543
3,2023-2056234,T-CROSS,2R2R,DHS,PCEQS114,2020297,ZP5,CINTO DE SEGURANÇA - FIXAÇÃO ASSOALHO ROSCA NOK,2,2023-12-05 21:36:33.284
4,2023-2056234,T-CROSS,2R2R,DHS,PCEQS620,2058570,ZP6,MÓDULO DO ACC (13) PEÇA INCORRETA ¨,9830946,2024-05-20 20:04:55.534


In [7]:
# Verificando os valores únicos da coluna MOTOR
print(df_atualizado['MOTOR'].unique())


['DHS' 'CWL' 'CWS' 'DRP' '   ']


In [8]:
# Visto que há poucos exemplares com o motor vazio (mais precisamente, 14 valores), é coerente a sua remoção 
df_one_hot = df_atualizado.drop(df_atualizado[df_atualizado['MOTOR'] == '   '].index)

In [9]:
#Checando para ver se há algum outro modelo de carro além do T-Cross
df_one_hot['MODELO'].unique()

array(['T-CROSS'], dtype=object)

In [10]:
#Checando para ver quais cores possuímos no dataset
df_one_hot['COR'].unique()

array(['2R2R', '0Q0Q', 'K2K2', '5T5T', 'A1A1', 'K2A1', '2RA1', '5TA1',
       '6K6K', '0QA1', '6KA1', '6UA1', 'O7O7'], dtype=object)

In [11]:
#Checando para ver quais tipos de motores possuímos no dataset
df_one_hot['MOTOR'].unique()

array(['DHS', 'CWL', 'CWS', 'DRP'], dtype=object)

In [12]:
#Tirando as colunas que não vamos utilizar

#A coluna "falha" não é importante porque todos os carros dessa tabela possuem falhas, então a informação valiosa é o tipo de falha.
df_one_hot.drop('FALHA', axis=1, inplace=True)

#A coluna "modelo" não é importante porque todos os carros dessa tabela possuem o mesmo modelo.
df_one_hot.drop('MODELO', axis=1, inplace=True)

#A coluna "estacao" não será utilizada porque antes da falha ser identificada, ainda não sabemos em qual estação o carro está.
df_one_hot.drop('ESTACAO', axis=1, inplace=True)

#A coluna "halle" não será utilizada porque, assim como a estação, ainda não sabemos em qual halle o carro está.
df_one_hot.drop('HALLE', axis=1, inplace=True)

#O usuário que cadastrou a falha não é relevante para a predição.
df_one_hot.drop('USUARIO', axis=1, inplace=True)

df_one_hot.head()

,KNR,COR,MOTOR,S_GROUP_ID,DATA
0,2023-2056234,2R2R,DHS,5,2024-05-20 21:31:51.992
1,2023-2056234,2R2R,DHS,4,2024-05-20 21:30:22.772
2,2023-2056234,2R2R,DHS,4,2023-12-06 06:48:29.543
3,2023-2056234,2R2R,DHS,2,2023-12-05 21:36:33.284
4,2023-2056234,2R2R,DHS,9830946,2024-05-20 20:04:55.534


In [13]:
# Mudando a coluna S_GROUP_ID usando o one hot encoding
df_one_hot = pd.get_dummies(df_one_hot, columns=['S_GROUP_ID'])
df_one_hot.head()

,KNR,COR,MOTOR,DATA,S_GROUP_ID_-2,S_GROUP_ID_1,S_GROUP_ID_2,S_GROUP_ID_4,S_GROUP_ID_5,S_GROUP_ID_133,S_GROUP_ID_137,S_GROUP_ID_140,S_GROUP_ID_9830946
0,2023-2056234,2R2R,DHS,2024-05-20 21:31:51.992,False,False,False,False,True,False,False,False,False
1,2023-2056234,2R2R,DHS,2024-05-20 21:30:22.772,False,False,False,True,False,False,False,False,False
2,2023-2056234,2R2R,DHS,2023-12-06 06:48:29.543,False,False,False,True,False,False,False,False,False
3,2023-2056234,2R2R,DHS,2023-12-05 21:36:33.284,False,False,True,False,False,False,False,False,False
4,2023-2056234,2R2R,DHS,2024-05-20 20:04:55.534,False,False,False,False,False,False,False,False,True


In [14]:
# Mudando a coluna S_GROUP_ID usando o one hot encoding
columns = ['S_GROUP_ID_-2','S_GROUP_ID_1','S_GROUP_ID_2','S_GROUP_ID_4','S_GROUP_ID_5','S_GROUP_ID_133','S_GROUP_ID_137','S_GROUP_ID_140','S_GROUP_ID_9830946']
for column in columns:
    df_one_hot[column] = df_one_hot[column].map({True: 1 ,  False: 0 })
df_one_hot.head()

,KNR,COR,MOTOR,DATA,S_GROUP_ID_-2,S_GROUP_ID_1,S_GROUP_ID_2,S_GROUP_ID_4,S_GROUP_ID_5,S_GROUP_ID_133,S_GROUP_ID_137,S_GROUP_ID_140,S_GROUP_ID_9830946
0,2023-2056234,2R2R,DHS,2024-05-20 21:31:51.992,0,0,0,0,1,0,0,0,0
1,2023-2056234,2R2R,DHS,2024-05-20 21:30:22.772,0,0,0,1,0,0,0,0,0
2,2023-2056234,2R2R,DHS,2023-12-06 06:48:29.543,0,0,0,1,0,0,0,0,0
3,2023-2056234,2R2R,DHS,2023-12-05 21:36:33.284,0,0,1,0,0,0,0,0,0
4,2023-2056234,2R2R,DHS,2024-05-20 20:04:55.534,0,0,0,0,0,0,0,0,1


In [16]:
# Agrupar por KNR, mantendo a primeira ocorrência de COR e MOTOR (já que o carro é o mesmo então essas informações se repetem em todos os registros) e somando as colunas S_GROUP_ID, para termos a noção de quantas repeitções de cada tipo de falha ocorreram para cada carro.

grouped_df = df_one_hot.groupby('KNR').agg({
    'COR': 'first',
    'MOTOR': 'first',
    'S_GROUP_ID_-2': 'sum',
    'S_GROUP_ID_1': 'sum',
    'S_GROUP_ID_2': 'sum',
    'S_GROUP_ID_4': 'sum',
    'S_GROUP_ID_5': 'sum',
    'S_GROUP_ID_133': 'sum',
    'S_GROUP_ID_137': 'sum',
    'S_GROUP_ID_140': 'sum',
    'S_GROUP_ID_9830946': 'sum'
}).reset_index()

# Exibir o DataFrame agrupado
grouped_df.head()


,KNR,COR,MOTOR,S_GROUP_ID_-2,S_GROUP_ID_1,S_GROUP_ID_2,S_GROUP_ID_4,S_GROUP_ID_5,S_GROUP_ID_133,S_GROUP_ID_137,S_GROUP_ID_140,S_GROUP_ID_9830946
0,2023-2056234,2R2R,DHS,0,1,1,2,1,0,0,0,1
1,2023-2316417,0Q0Q,DHS,1,0,0,0,0,0,0,0,0
2,2023-2316420,0Q0Q,DHS,1,0,0,0,0,0,0,0,0
3,2023-3016123,2R2R,CWL,0,0,3,1,0,0,0,0,0
4,2023-3016194,0Q0Q,DHS,0,4,7,2,0,0,0,0,0


In [17]:
#Exportando o DataFrame agrupado para um arquivo CSV
grouped_df.to_csv('../data/Falhas_categorizadas.csv', index=False)

In [19]:
#Juntando com as informações dos KNRS que temos na tabela de resultados
df_resultados = pd.read_csv('../data/df_resultados_tratado.csv')

#Juntando os dois DataFrames e tirando as linhas que possuem valores nulos
df_final = pd.merge(df_resultados, grouped_df, on='KNR')
df_final = df_final.dropna()
df_final.head()

,KNR,ID1NAME,ID1SOK,ID1SNOK,ID1DATA,ID2NAME,ID2SOK,ID2SNOK,ID2DATA,ID718NAME,...,MOTOR,S_GROUP_ID_-2,S_GROUP_ID_1,S_GROUP_ID_2,S_GROUP_ID_4,S_GROUP_ID_5,S_GROUP_ID_133,S_GROUP_ID_137,S_GROUP_ID_140,S_GROUP_ID_9830946
0,2023-2056234,5.0,5.0,0.0,0.0,770.0,766.0,4.0,4.491227,71.0,...,DHS,0,1,1,2,1,0,0,0,1
1,2023-3016123,2.0,2.0,0.0,0.0,838.0,838.0,0.0,5.236377,37.0,...,CWL,0,0,3,1,0,0,0,0,0
2,2023-3016194,2.0,2.0,0.0,0.0,852.0,844.0,8.0,5.960012,46.0,...,DHS,0,4,7,2,0,0,0,0,0
3,2023-3026027,2.0,2.0,0.0,0.0,854.0,848.0,6.0,5.831898,36.0,...,DHS,0,2,7,3,0,0,0,0,0
4,2023-3026134,2.0,2.0,0.0,0.0,834.0,834.0,0.0,6.207396,36.0,...,CWL,0,1,4,4,2,0,0,0,0


In [20]:
#features do df final
df_final.columns

Index(['KNR', 'ID1NAME', 'ID1SOK', 'ID1SNOK', 'ID1DATA', 'ID2NAME', 'ID2SOK',
       'ID2SNOK', 'ID2DATA', 'ID718NAME', 'ID718SOK', 'ID718SNOK', 'ID718DATA',
       'COR', 'MOTOR', 'S_GROUP_ID_-2', 'S_GROUP_ID_1', 'S_GROUP_ID_2',
       'S_GROUP_ID_4', 'S_GROUP_ID_5', 'S_GROUP_ID_133', 'S_GROUP_ID_137',
       'S_GROUP_ID_140', 'S_GROUP_ID_9830946'],
      dtype='object')

In [21]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40154 entries, 0 to 40153
Data columns (total 24 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   KNR                 40154 non-null  object 
 1   ID1NAME             40154 non-null  float64
 2   ID1SOK              40154 non-null  float64
 3   ID1SNOK             40154 non-null  float64
 4   ID1DATA             40154 non-null  float64
 5   ID2NAME             40154 non-null  float64
 6   ID2SOK              40154 non-null  float64
 7   ID2SNOK             40154 non-null  float64
 8   ID2DATA             40154 non-null  float64
 9   ID718NAME           40154 non-null  float64
 10  ID718SOK            40154 non-null  float64
 11  ID718SNOK           40154 non-null  float64
 12  ID718DATA           40154 non-null  float64
 13  COR                 40154 non-null  object 
 14  MOTOR               40154 non-null  object 
 15  S_GROUP_ID_-2       40154 non-null  int64  
 16  S_GR

É possível observar que há ainda duas colunas com o formato "object", que muitos modelos de rede neural não conseguem processar. Por esse motivo, vamos transformar cada categoria de cor e motor em colunas usando o One-hot-encoding.

In [34]:
#Transformando as colunas COR e MOTOR em colunas numéricas
#df_final = pd.get_dummies(df_final, columns=['COR', 'MOTOR'])

#Substituindo os valores das colunas 'COR_0Q0Q', 'COR_0QA1', 'COR_2R2R', 'COR_2RA1','COR_5T5T', 'COR_5TA1', 'COR_6K6K', 'COR_6KA1','COR_6UA1', 'COR_A1A1','COR_K2A1', 'COR_K2K2', 'MOTOR_CWL', 'MOTOR_CWS', 'MOTOR_DHS', 'MOTOR_DRP' por 0 e 1

columns_to_convert = [
    'COR_0Q0Q', 'COR_0QA1', 'COR_2R2R', 'COR_2RA1', 'COR_5T5T', 
    'COR_5TA1', 'COR_6K6K', 'COR_6KA1', 'COR_6UA1', 'COR_A1A1',
    'COR_K2A1', 'COR_K2K2', 'MOTOR_CWL', 'MOTOR_CWS', 'MOTOR_DHS', 'MOTOR_DRP'
]

# Converter as colunas de booleano para 0 e 1
df_final[columns_to_convert] = df_final[columns_to_convert].astype(int)

df_final.head()

,KNR,ID1NAME,ID1SOK,ID1SNOK,ID1DATA,ID2NAME,ID2SOK,ID2SNOK,ID2DATA,ID718NAME,...,COR_6K6K,COR_6KA1,COR_6UA1,COR_A1A1,COR_K2A1,COR_K2K2,MOTOR_CWL,MOTOR_CWS,MOTOR_DHS,MOTOR_DRP
0,2023-2056234,5.0,5.0,0.0,0.0,770.0,766.0,4.0,4.491227,71.0,...,0,0,0,0,0,0,0,0,1,0
1,2023-3016123,2.0,2.0,0.0,0.0,838.0,838.0,0.0,5.236377,37.0,...,0,0,0,0,0,0,1,0,0,0
2,2023-3016194,2.0,2.0,0.0,0.0,852.0,844.0,8.0,5.960012,46.0,...,0,0,0,0,0,0,0,0,1,0
3,2023-3026027,2.0,2.0,0.0,0.0,854.0,848.0,6.0,5.831898,36.0,...,0,0,0,0,0,0,0,0,1,0
4,2023-3026134,2.0,2.0,0.0,0.0,834.0,834.0,0.0,6.207396,36.0,...,0,0,0,0,0,0,1,0,0,0


In [35]:
#Quantos registros temos no df_final
df_final.shape


(40154, 38)

In [36]:
#Verificando quantos valores únicos de KNR temos no df_final
df_final['KNR'].nunique()

40154

In [39]:
#Ultima checaagem para ver se há valores nulos
df_final.isnull().sum()

KNR                   0
ID1NAME               0
ID1SOK                0
ID1SNOK               0
ID1DATA               0
ID2NAME               0
ID2SOK                0
ID2SNOK               0
ID2DATA               0
ID718NAME             0
ID718SOK              0
ID718SNOK             0
ID718DATA             0
S_GROUP_ID_-2         0
S_GROUP_ID_1          0
S_GROUP_ID_2          0
S_GROUP_ID_4          0
S_GROUP_ID_5          0
S_GROUP_ID_133        0
S_GROUP_ID_137        0
S_GROUP_ID_140        0
S_GROUP_ID_9830946    0
COR_0Q0Q              0
COR_0QA1              0
COR_2R2R              0
COR_2RA1              0
COR_5T5T              0
COR_5TA1              0
COR_6K6K              0
COR_6KA1              0
COR_6UA1              0
COR_A1A1              0
COR_K2A1              0
COR_K2K2              0
MOTOR_CWL             0
MOTOR_CWS             0
MOTOR_DHS             0
MOTOR_DRP             0
dtype: int64

In [38]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40154 entries, 0 to 40153
Data columns (total 38 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   KNR                 40154 non-null  object 
 1   ID1NAME             40154 non-null  float64
 2   ID1SOK              40154 non-null  float64
 3   ID1SNOK             40154 non-null  float64
 4   ID1DATA             40154 non-null  float64
 5   ID2NAME             40154 non-null  float64
 6   ID2SOK              40154 non-null  float64
 7   ID2SNOK             40154 non-null  float64
 8   ID2DATA             40154 non-null  float64
 9   ID718NAME           40154 non-null  float64
 10  ID718SOK            40154 non-null  float64
 11  ID718SNOK           40154 non-null  float64
 12  ID718DATA           40154 non-null  float64
 13  S_GROUP_ID_-2       40154 non-null  int64  
 14  S_GROUP_ID_1        40154 non-null  int64  
 15  S_GROUP_ID_2        40154 non-null  int64  
 16  S_GR

In [37]:
#Exportando o DataFrame final para um arquivo CSV
df_final.to_csv('../data/Merge_Falhas_Resultados.csv', index=False)

## Conclusão da preparação de dados

Ao final dessa manipulação dos dados, temos uma tabela final que reúne todos os acontecimentos na linha de produção do carro (conforme a tabela tratada de resultados que veio do notebook *main.py*) e quais falhas foram encontradas.

Vale ressaltar que o objetivo de uso dessa tabela e auxiliar os modelos a prever qual é o tipo de falha que haverá num carro, por isso todos os carros nesse conjunto de dados possuem falhas.

